# Realsense Python Demo
Imports:

In [ ]:
import pyrealsense2 as rs
import numpy as np
from enum import IntEnum
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd

# from RealSenseGUI import draw_geometries
# o3d.visualization.draw_geometries = draw_geometries

Definitions:

In [ ]:
class Preset(IntEnum): # Preset for the RealSense Firmware
    Custom = 0
    Default = 1
    Hand = 2
    HighAccuracy = 3
    HighDensity = 4
    MediumDensity = 5

# Calculate Intrinsic Camera parameters of the taken frame
def get_intrinsic_matrix(frame):
    intrinsics = frame.profile.as_video_stream_profile().intrinsics
    out = o3d.camera.PinholeCameraIntrinsic(640, 480, intrinsics.fx,
                                            intrinsics.fy, intrinsics.ppx,
                                            intrinsics.ppy)
    return out

# Matrix to transform Pointcloud
flip_transform = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]
# Saves all pointClouds before processing
pointClouds = []

Setup Camera:

In [ ]:
# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
# different resolutions of color and depth streams
config = rs.config()

config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.rgb8, 30)

# Start streaming
profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()

# Set Depth Sensor Firmware Preset
depth_sensor.set_option(rs.option.visual_preset, Preset.Custom)
depth_scale = depth_sensor.get_depth_scale()

# Maximum Depth
clipping_distance_in_meters = 2  
clipping_distance = clipping_distance_in_meters / depth_scale

# Performs alignment between depth image and color image
align_to = rs.stream.color
align = rs.align(align_to)


Capture Frames:

In [ ]:
for _ in range(10): # Capture 10 PointClouds

    # Get frameset of color and depth
    frames = pipeline.wait_for_frames()

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    # Get aligned frames
    aligned_depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        get_intrinsic_matrix(color_frame))

    # Validate that both frames are valid
    if not aligned_depth_frame or not color_frame:
        continue
    
    # Create Open3D Images
    depth_image = o3d.geometry.Image(
        np.array(aligned_depth_frame.get_data()))
    color_temp = np.asarray(color_frame.get_data())
    color_image = o3d.geometry.Image(color_temp)

    # Create RGBD Image
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_image,
        depth_image,
        depth_scale=1.0 / depth_scale,
        depth_trunc=clipping_distance_in_meters,
        convert_rgb_to_intensity=False)
    
    # Convert to PointCloud
    temp = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, intrinsic)
    temp.transform(flip_transform)
    pointClouds.append(temp)

Combine PointClouds:

In [ ]:
pcd = o3d.geometry.PointCloud()
for cloud in pointClouds:
    pcd += cloud

In [ ]:
pcd = o3d.io.read_point_cloud("rawPointcloud.pcd") # Worst Case

Visualize PointCloud:

In [ ]:
o3d.visualization.draw_geometries([pcd])

Downsample & Cluster Image:

In [ ]:
pcd = pcd.voxel_down_sample(voxel_size=0.008)
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    # Label all Points into clusters
    labels = np.array(
        pcd.cluster_dbscan(eps=0.08, min_points=120, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Set Colors of Points
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

Visualize PointCloud:

In [ ]:
o3d.visualization.draw_geometries([pcd])

Find Closest Point:

In [ ]:
pcdNP = np.asarray(pcd.points)
clusterClosestPoint = [[0,1e10] for _ in range(max_label+2)]
for index, label in enumerate(labels):
    # if label == 0 :
    #     continue
    if clusterClosestPoint[label][1]>np.linalg.norm(pcdNP[index]):
        clusterClosestPoint[label][0] = index
        clusterClosestPoint[label][1] = np.linalg.norm(pcdNP[index])
        
pd.DataFrame(clusterClosestPoint, columns=["Point Index", "Distance(m)"])

Label Visualization:

In [ ]:
points = [[0,0,0]]
lines = []
textLabels = []
for cluster, (pcdIndex, distance) in enumerate(clusterClosestPoint):
    textLabels.append([pcdNP[pcdIndex].tolist(),f"Cluster {cluster+1}"])
    points.append(pcdNP[pcdIndex].tolist())
    lines.append([0,cluster+1])
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)

Visualize PointCloud:

In [ ]:
o3d.visualization.draw_geometries([pcd,line_set])